In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

五个类别+others

In [ ]:
import torch
import clip
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载模型
model, preprocess = clip.load("ViT-B/32", device=device)

# 定义类别和对应的图片命名规则
categories = {
    "pet cat": "宠物猫",
    "tomato": "番茄",
    "paper-cut": "剪纸",
    "computer": "电脑",
    "dumpling": "饺子"
}

# 初始化统计数据
stats = {category: {"TP": 0, "FP": 0, "FN": 0} for category in categories.keys()}

# 图片文件夹路径
image_folder_path = "/content/drive/MyDrive/picture_data"

# 遍历图片
for image_name in os.listdir(image_folder_path):
    if image_name.endswith(".png"):
        image_path = os.path.join(image_folder_path, image_name)
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        text = clip.tokenize(list(categories.keys()) + ["others"]).to(device)
        with torch.no_grad():
            logits_per_image, _ = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        # with torch.no_grad():
        #   image_features = model.encode_image(image)
        #   text_features = model.encode_text(text)

        # image_features /= image_features.norm(dim=-1, keepdim=True)
        # text_features /= text_features.norm(dim=-1, keepdim=True)

        # similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        # probs = similarity.cpu().numpy()

        predicted_category = (list(categories.keys()) + ["others"])[probs.argmax()]

        category_matched = False
        for category, prefix in categories.items():
            if prefix in image_name:
                category_matched = True
                if category == predicted_category:
                    stats[category]["TP"] += 1
                else:
                    stats[category]["FN"] += 1
                    if predicted_category != "others":
                        stats[predicted_category]["FP"] += 1
                break

        # 处理负样本
        if not category_matched:
            if predicted_category != "others":
                stats[predicted_category]["FP"] += 1  # 错误地将负样本判断为正样本类别

# 计算平均精确度、平均召回率以及平均F1分数
total_precision = 0
total_recall = 0
total_f1 = 0
num_categories = len(categories)

for category, data in stats.items():
    if category != "others":
        precision = data["TP"] / (data["TP"] + data["FP"]) if (data["TP"] + data["FP"]) > 0 else 0
        recall = data["TP"] / (data["TP"] + data["FN"]) if (data["TP"] + data["FN"]) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        total_precision += precision
        total_recall += recall
        total_f1 += f1

average_precision = total_precision / num_categories
average_recall = total_recall / num_categories
average_f1 = total_f1 / num_categories

print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Category: pet cat, Precision: 0.9256, Recall: 0.9950, F1: 0.9590
Category: tomato, Precision: 0.4785, Recall: 1.0000, F1: 0.6472
Category: paper-cut, Precision: 0.5535, Recall: 0.9050, F1: 0.6869
Category: computer, Precision: 0.5319, Recall: 1.0000, F1: 0.6944
Category: dumpling, Precision: 0.3333, Recall: 0.9900, F1: 0.4987
Average Precision: 0.5646, Average Recall: 0.9780, Average F1: 0.6973


一个类别+others

In [ ]:
total_precision = 0
total_recall = 0
total_f1 = 0

# 对每个类别分别计算precision、recall及F1分数
for category, keyword in categories.items():
    # 初始化统计数据
    TP = 0
    FP = 0
    FN = 0

    # 遍历图片
    for image_name in os.listdir(image_folder_path):
        if image_name.endswith(".png"):
            image_path = os.path.join(image_folder_path, image_name)

            # 预处理图片并进行预测
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
            text_labels = [category] + ["others"]
            text = clip.tokenize(text_labels).to(device)

            with torch.no_grad():
                logits_per_image, _ = model(image, text)
                probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            # 获取最高概率的类别
            predicted_category = text_labels[probs.argmax()]

            # 判断真实类别并更新统计数据
            actual_category = "others" if keyword not in image_name else category
            if predicted_category == actual_category and actual_category == category:
                TP += 1
            elif predicted_category == category and actual_category == "others":
                FP += 1
            elif predicted_category == "others" and actual_category == category:
                FN += 1

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score

# 计算并打印平均Precision、Recall及F1分数
average_precision = total_precision / len(categories)
average_recall = total_recall / len(categories)
average_f1 = total_f1 / len(categories)

print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")

Category: pet cat, Precision: 0.5952, Recall: 1.0000, F1 Score: 0.7463
Category: tomato, Precision: 0.2217, Recall: 1.0000, F1 Score: 0.3630
Category: paper-cut, Precision: 0.3306, Recall: 1.0000, F1 Score: 0.4969
Category: computer, Precision: 0.4598, Recall: 1.0000, F1 Score: 0.6299
Category: dumpling, Precision: 0.2051, Recall: 1.0000, F1 Score: 0.3404
Average Precision: 0.3625, Average Recall: 1.0000, Average F1: 0.5153


卡阈值（单类别+others）

In [ ]:
# 设定阈值
threshold = 0.97

total_precision = 0
total_recall = 0
total_f1 = 0

# 对每个类别分别计算precision和recall
for category, keyword in categories.items():
    # 初始化统计数据
    TP = 0
    FP = 0
    FN = 0

    # 遍历图片
    for image_name in os.listdir(image_folder_path):
        if image_name.endswith(".png"):
            image_path = os.path.join(image_folder_path, image_name)

            # 预处理图片
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

            # 准备文本
            text_labels = [category, "others"]
            text = clip.tokenize(text_labels).to(device)

            # 进行预测
            with torch.no_grad():
                logits_per_image, _ = model(image, text)
                probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            # 使用阈值判断类别
            is_positive_prediction = probs[0, 0] > threshold
            predicted_category = category if is_positive_prediction else "others"

            # 判断真实类别
            actual_category = category if keyword in image_name else "others"

            # 更新统计数据
            if predicted_category == actual_category and actual_category == category:
                TP += 1
            elif predicted_category == category and actual_category == "others":
                FP += 1
            elif actual_category == category and predicted_category == "others":
                FN += 1

    # 计算Precision和Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


    print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score

# 计算并打印平均Precision和Recall
average_precision = total_precision / len(categories)
average_recall = total_recall / len(categories)
average_f1 = total_f1 / len(categories)

print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")

Category: pet cat, Precision: 0.9789, Recall: 0.9300, F1 Score: 0.9538
Category: tomato, Precision: 0.6923, Recall: 0.9900, F1 Score: 0.8148
Category: paper-cut, Precision: 0.8058, Recall: 0.5600, F1 Score: 0.6608
Category: computer, Precision: 0.8428, Recall: 0.9650, F1 Score: 0.8998
Category: dumpling, Precision: 0.4910, Recall: 0.9600, F1 Score: 0.6497
Average Precision: 0.7622, Average Recall: 0.8810, Average F1: 0.7958


卡阈值（单类别相似度）

In [ ]:
import torch
import clip
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载模型
model, preprocess = clip.load("ViT-B/32", device=device)

# 定义类别和对应的图片命名规则
categories = {
    "pet cat": "宠物猫",
    "tomato": "番茄",
    "paper-cut": "剪纸",
    "computer": "电脑",
    "dumpling": "饺子"
}

# 设定阈值
threshold = 0.23

total_precision = 0
total_recall = 0
total_f1 = 0

# 对每个类别分别计算precision和recall
for category, keyword in categories.items():
    # 初始化统计数据
    TP = 0
    FP = 0
    FN = 0

    # 遍历图片
    for image_name in os.listdir(image_folder_path):
        if image_name.endswith(".png"):
            image_path = os.path.join(image_folder_path, image_name)

            # 预处理图片
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

            # 准备文本
            text_labels = [category]
            text = clip.tokenize(text_labels).to(device)

            # 进行预测
            with torch.no_grad():
              image_features = model.encode_image(image)
              text_features = model.encode_text(text)

            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            similarity = image_features @ text_features.T

            # 使用阈值判断类别
            is_positive_prediction = similarity[0] > threshold
            predicted_category = category if is_positive_prediction else "others"

            # 判断真实类别
            actual_category = category if keyword in image_name else "others"

            # 更新统计数据
            if predicted_category == actual_category and actual_category == category:
                TP += 1
            elif predicted_category == category and actual_category == "others":
                FP += 1
            elif actual_category == category and predicted_category == "others":
                FN += 1

    # 计算Precision和Recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


    print(f"Category: {category}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score

# 计算并打印平均Precision和Recall
average_precision = total_precision / len(categories)
average_recall = total_recall / len(categories)
average_f1 = total_f1 / len(categories)

print(f"Average Precision: {average_precision:.4f}, Average Recall: {average_recall:.4f}, Average F1: {average_f1:.4f}")

Category: pet cat, Precision: 0.9132, Recall: 1.0000, F1 Score: 0.9547
Category: tomato, Precision: 0.5038, Recall: 0.9850, F1 Score: 0.6667
Category: paper-cut, Precision: 0.7036, Recall: 0.8900, F1 Score: 0.7859
Category: computer, Precision: 0.9031, Recall: 0.8850, F1 Score: 0.8939
Category: dumpling, Precision: 0.3186, Recall: 0.9700, F1 Score: 0.4796
Average Precision: 0.6685, Average Recall: 0.9460, Average F1: 0.7561
